In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

rc('font', family='AppleGothic')
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

warnings.filterwarnings(action='ignore')

In [2]:
import argparse
import random as rn
import tensorflow as tf
import keras.layers as L
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
from keras import backend as K
from keras.models import load_model,Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing import sequence
from keras.utils.data_utils import Sequence
from keras.regularizers import l2

In [3]:
#재현성을 위한 seed 설정
seed_num = 42
np.random.seed(seed_num)
rn.seed(seed_num)
tf.random.set_seed(seed_num)

In [4]:
data = pd.read_csv('./raw_data/final_day.csv', index_col = 0)
weathers = ['평균기온', '일강수량', '일사량', '미세먼지'] # 종속변수
products = data.columns[4:].to_list()

In [5]:
weathers = ['평균기온', '일강수량', '미세먼지'] # 종속변수
products = ['기능성 모공관리 화장품', '냉풍기', '벽걸이 에어컨']

# Models

### 1. OLS

In [6]:
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson


def OLS_(data, cat):
    X = data[weathers]
    y = data[cat]
    
    model = sm.OLS(y, X).fit()
    mse = mean_squared_error(y_true=y, y_pred = model.predict(X))

    return model, model.rsquared, mse, durbin_watson(model.resid) 
    # 모델, R제곱, mse, DW지수

### 2. LSTM

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU

In [8]:
def build_data(data, cat):
    cols = [cat]+ weathers
    X = data[cols].reset_index(drop=False)
    X.rename(columns={cat:'y'}, inplace=True)

    X.index = X['date']
    del X['date']

    return X

In [9]:
def minmax_scalar(X):
    idx = X.index
    col = X.columns

    scalar = MinMaxScaler()
    scaled_X = pd.DataFrame(scalar.fit_transform(X))
    scaled_X.index = idx
    scaled_X.columns = col

    return scaled_X

In [10]:
def split_xy(dataset, time_steps, y_column): 
    
    x, y = list(), list()
    
    for i in range(len(dataset)): # 2년치 일별데이터면 730번 for문 실행
        x_end_number = i + time_steps
        y_end_number = x_end_number + y_column

        if y_end_number > len(dataset): # 데이터 끝에 다다르면 끝
            break

        tmp_x = np.array(dataset)[i:x_end_number, 1:] # 1:으로 수정(y칼럼 제외)
        tmp_y = np.array(dataset)[x_end_number:y_end_number, 0] # 0으로 수정(y칼럼)
        
        x.append(tmp_x)
        y.append(tmp_y)
        
    return np.array(x), np.array(y)

In [11]:
# def RMSLE_fun(origin, pred):
#     rmsle = np.sqrt(mean_squared_log_error(origin+1, pred+1))
#     return rmsle

In [12]:
def data_pipeline(data, cat, time_steps, y_columns):
    data = build_data(data, cat)

    min = data['y'].min()
    max = data['y'].max()

    X = minmax_scalar(data)
    Xy = X.dropna()

    X, y = split_xy(Xy, time_steps, y_columns)

    X_train, y_train = X[:-7],y[:-7]
    X_test, y_test = X[-7:],y[-7:]

    # X_train.shape[2] = feature 개수
    X_test=X_test.reshape(-1, time_steps, X_train.shape[2]) 
    y_test=y_test.reshape(-1, y_columns)


    return X_train, y_train, X_test, y_test, min, max

In [13]:
def LSTM_(data, cat, timestep=7, y_column=1):
    X_train,y_train,X_test,y_test,min,max = data_pipeline(data, cat, timestep, y_column)
    
    model = Sequential()
    model.add(LSTM(128, input_shape = (None, X_train.shape[2])))
    model.add(Dense(16))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
    
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stopping], validation_data = (X_test, y_test))

    y_pred = model.predict(X_test, batch_size=1)
    r2 = r2_score(y_test, y_pred)
    mse = np.mean((y_test-y_pred)**2)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(y_test, y_pred)
#     rmsle = RMSLE_fun(np.array(y_test), np.array(y_pred))
    
    return model, r2, mse

### 3. GRU

In [14]:
def GRU_(data, cat, timestep=7, y_column=1):
    
    X_train,y_train,X_test,y_test,min,max = data_pipeline(data, cat, timestep, y_column)
    
    model = Sequential()
    model.add(GRU(128, input_shape = (None, X_train.shape[2])))
    model.add(Dense(16))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stopping], validation_data = (X_test, y_test))

    y_pred = model.predict(X_test, batch_size=1)
    r2 = r2_score(y_test, y_pred)
    mse = np.mean((y_test-y_pred)**2)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(y_test, y_pred)
#     rmsle = RMSLE_fun(np.array(y_test), np.array(y_pred))
    
    return model, r2, mse

### 4. RETAIN

In [15]:
def RETAIN_(data, cat, timesteps=7, y_columns=1, epochs=100):
    X_train, y_train, X_test, y_test, min, max = data_pipeline(data, cat, timesteps, y_columns)

    def reshape(data):
        return K.reshape(x=data, shape=(-1, 1, 3)) # (-1, 1, feature의 개수)

    input = keras.layers.Input(shape =(timesteps, X_train.shape[2]), name='input') # (타임스텝, feature개수)

    alpha = keras.layers.Bidirectional(keras.layers.LSTM(X_train.shape[2],
                                  return_sequences=True, implementation=2),
                                  name='alpha')
    beta = keras.layers.Bidirectional(keras.layers.LSTM(X_train.shape[2],
                                  return_sequences=True, implementation=2),
                                  name='beta')

    alpha_dense = keras.layers.Dense(1)
    beta_dense = keras.layers.Dense(X_train.shape[2])

    #Compute alpha, visit attention
    alpha_out = alpha(input)
    alpha_out = keras.layers.TimeDistributed(alpha_dense, name='alpha_dense_0')(alpha_out)
    alpha_out = keras.layers.Softmax(axis=1,name='softmax_1')(alpha_out)

    #Compute beta, codes attention
    beta_out = beta(input)
    beta_out = keras.layers.TimeDistributed(beta_dense)(beta_out)
    beta_out = keras.layers.Activation('tanh',name='beta_dense_0')(beta_out)
    #Compute context vector based on attentions and embeddings

    c_t = keras.layers.Multiply()([alpha_out, beta_out, input])
    c_t = keras.layers.Lambda(lambda x: K.sum(x, axis=1))(c_t)
    #Reshape to 3d vector for consistency between Many to Many and Many to One implementations
    contexts = keras.layers.Lambda(reshape)(c_t)

    #Make a prediction
    contexts = keras.layers.Dropout(0.1)(contexts)
    output_layer = keras.layers.Dense(1, name='dOut', activation = 'linear') 

    #TimeDistributed is used for consistency
    # between Many to Many and Many to One implementations
    output = keras.layers.TimeDistributed(output_layer, name='time_distributed_out')(contexts)
    #Define the model with appropriate inputs
    
    model = Model(inputs=input, outputs=[output])
    model.compile(optimizer='adam', loss='mean_squared_error', sample_weight_mode="temporal",metrics=['mse', 'mae', 'mape'])

    early_stopping = EarlyStopping(monitor='mse', patience=10, mode='min',restore_best_weights=True)

    # 모델 저장 : 학습된 모델 개별 저장함. callbacks에 modelsaver 변수 추가시 저장가능
    # modelsaver = ModelCheckpoint("./models/{}_retain.hdf5".format(cat), monitor = 'val_loss', mode = 'min', verbose=1, save_best_only=True)

    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0, callbacks=[early_stopping], validation_data = (X_test,y_test))

    y_pred = model.predict(X_test, batch_size=1)  
    y_test = y_test.reshape(-1,1) *(max-min)+min
    y_pred =y_pred.reshape(-1,1) *(max-min)+min

    y_train_pred = model.predict(X_train, batch_size=1)
    y_train_pred = y_train_pred.reshape(-1,1)
    y_train_test = y_train.reshape(-1,1)

    r2 = r2_score(np.array(y_test),np.array(y_pred)) # R2값 추가하였음
#     rmsle = RMSLE_fun(np.array(y_test),np.array(y_pred))
    mse = mean_squared_error(np.array(y_test),np.array(y_pred))
#     rmse = np.sqrt(mse)
    
    return model, r2, mse

### ADF test

In [16]:
from statsmodels.tsa.stattools import  adfuller

def ADF_(timeseries):
    dftest = adfuller(timeseries, autolag='AIC', maxlag = 20)
    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistic', 'p-value',  '#Lags Used', 'Number of Observations used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value(%s)'%key] = value
    pvalue = dftest[1]
    if pvalue < 0.05:
#         print('p-value = %.4f. The series is likely stationary.' % pvalue)
        return True
    else:
#         print('p-vale = %.4f. The series is likely non-stationary.' % pvalue)
        return False

### Diff

In [17]:
def DIFF_(timeseries):
    timeseries = timeseries - timeseries.shift()
    timeseries = timeseries.dropna()
    return timeseries

### 날씨만 미리 차분

In [18]:
data_diff = data.copy() # 날씨 차분된 데이터

for weather in weathers:
    timeseries = data[weather]
    if ADF_(timeseries) == False:
        print(weather)
        data_diff[weather] = DIFF_(timeseries)
        
data_diff

평균기온


,평균기온,일강수량,일사량,미세먼지,기능성 링클케어 화장품,기능성 모공관리 화장품,기능성 아이케어 화장품,기능성 영양보습 화장품,기능성 트러블케어 화장품,기능성 화이트닝 화장품,...,제습기,중대형 에어컨,천장형 에어컨,초음파식 가습기,카페트매트,컨벡터,탁상/USB 선풍기,황토매트,휴대용 선풍기,히터
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,NaN,0.000000,9.202857,32.962963,3,20,364,13,44,0,...,24,1,0,252,11,19,0,4,1,119
2018-01-02,0.551926,0.000000,7.342857,40.037037,31,38,324,24,47,0,...,24,0,0,399,15,18,4,6,4,287
2018-01-03,-1.730873,1.900000,9.082619,23.185185,11,40,415,11,66,0,...,16,0,1,412,9,29,2,7,1,284
2018-01-04,-0.178947,1.916667,6.590238,26.423077,2,27,657,11,51,1,...,7,0,0,494,12,30,1,7,3,242
2018-01-05,1.596842,1.160000,7.451667,29.642857,5,28,643,5,52,0,...,15,0,0,320,15,19,3,7,3,242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,-3.054737,0.535714,9.605909,22.655172,8,44,260,13,48,0,...,5,0,0,271,8,20,3,3,2,233
2019-12-28,-0.084211,0.500000,9.964091,32.862069,2,28,167,7,54,0,...,7,0,0,164,4,5,2,4,5,79
2019-12-29,2.231579,7.406667,3.125227,34.172414,9,47,1532,9,59,0,...,10,0,1,189,7,10,0,11,2,122


### 5. VAR

In [19]:
from statsmodels.tsa.api import VAR

In [20]:
def VAR_(data, cat):

    df = data[[cat]+['일강수량', '평균기온', '미세먼지', '일사량']]

    model = VAR(df)
    results_aic = []

    for p in range(1,10):
        results = model.fit(p)
        results_aic.append(results.aic)

    order = np.argmin(results_aic)+1 # AIC가 가장 작은 모델
    fitted = model.fit(order)

    forecast_input = df.values[-order:]
    forecast_input

    nobs = 10 # 학습, 검증 데이터 split 개수
    df_train, df_test = df[0:-nobs], df[-nobs:]

    fc = fitted.forecast(y=forecast_input, steps=nobs)
    df_forecast = pd.DataFrame(fc, index=df.index[-nobs:], columns=df.columns+'_2d')


    def invert_transformation(df_train, df_forecast, second_diff=False):
        """Revert back the differencing to get the forecast to original scale."""
        df_fc = df_forecast.copy()
        columns = df_train.columns
        for col in columns:        
            # Roll back 2nd Diff
            if second_diff:
                df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
            # Roll back 1st Diff
            df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
        return df_fc

    df_results = invert_transformation(df_train, df_forecast, second_diff=True)        
    df_results.loc[:, [cat+'_forecast'] + ['일강수량_forecast', '평균기온_forecast','미세먼지_forecast', '일사량_forecast']]

    mse = np.mean((df_results[cat+'_forecast'].values - df_test[cat])**2)

    return fitted, mse

### Granger test

In [21]:
# from statsmodels.tsa.stattools import grangercausalitytests

# maxlag=12

# test = 'ssr_chi2test'
# def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
#     """Check Granger Causality of all possible combinations of the Time series.
#     The rows are the response variable, columns are predictors. The values in the table 
#     are the P-Values. P-Values lesser than the significance level (0.05), implies 
#     the Null Hypothesis that the coefficients of the corresponding past values is 
#     zero, that is, the X does not cause Y can be rejected.

#     data      : pandas dataframe containing the time series variables
#     variables : list containing names of the time series variables.
#     """
#     df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
#     for c in df.columns:
#         for r in df.index:
#             test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
#             p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
#             if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
#             min_p_value = np.min(p_values)
#             df.loc[r, c] = min_p_value
#     df.columns = [var + '_x' for var in variables]
#     df.index = [var + '_y' for var in variables]
#     return df

# grangers_causation_matrix(df, variables = df.columns)  

# score comparison

(Cohen's)
 
small   : 0.02 ≤ R2 

middle : 0.13 ≤ R2

large   : 0.26 ≤ R2


* 일반적으로 0.65~0.7을 기준으로 하므로 엄격하게 기준을 설정할 수도 있음

In [ ]:
not_target = []
target = []

OLS_scores = []
LSTM_scores = []
RETAIN_scores = []
GRU_scores = []
VAR_scores = []

# OLS
for cat in products:
    try:
        ols_model, ols_r2, ols_mse, DW = OLS_(data, cat)
        if (ols_r2 < 0.65) and (1.5 <= DW <= 2.5): # OLS 결과도 좋지 않고, 시계열적이지도 않으면 날씨와 관련없다고 판단
            print(cat, ': not using this data')
            not_target.append(cat)
        else:
            target.append(cat)
            OLS_scores.append(ols_mse)
    except:
        OLS_scores.append(np.nan)

print('\n OLS finished \n')
        
# OLS로 한 차례 걸러진 데이터 타겟으로 딥러닝, 시계열 모델링 진행
for cat in target:
    print('================', cat, '=================')
    # deep learning
    try:
        LSTM_model, LSTM_r2, LSTM_mse = LSTM_(data, cat)
        LSTM_scores.append(LSTM_mse)
        print('LSTM 완료')
    except:
        LSTM_scores.append(np.nan)

    try:
        RETAIN_model, RETAIN_r2, RETAIN_mse = RETAIN_(data, cat)
        RETAIN_scores.append(RETAIN_mse)
        print('RETAIN 완료')
    except:
        RETAIN_scores.append(np.nan)

    try:
        GRU_model, GRU_r2, GRU_mse = GRU_(data, cat) 
        GRU_scores.append(GRU_mse)
        print('GRU 완료')
    except:
        GRU_scores.append(np.nan)

    # timeseries
    try:
        # 정상성 검정
        timeseries = data[[cat]]
        
        if ADF_(timeseries) == False:
            timeseries = DIFF_(timeseries) # 1차 차분
            print('1차 차분')
        if ADF_(timeseries) == False:
            timeseries = DIFF_(timeseries) # 2차 차분
            print('2차 차분')
        if ADF_(timeseries) == False:
            print("2차 차분으로 정상성 확보되지 않음 -> 확인 필요")
        
        # 차분된 날씨데이터에 대체
        data_diff[cat] = timeseries
        data_diff.dropna(inplace=True)
        VAR_model, VAR_mse = VAR_(data_diff, cat)
        VAR_scores.append(VAR_mse)
        print('VAR 완료')
        
    except:
        VAR_scores.append(np.nan)
    print('\n')


 OLS finished 

================ 기능성 모공관리 화장품 =================
LSTM 완료


In [ ]:
print('개수 일치해야 함')
print(len(OLS_scores), len(LSTM_scores), len(RETAIN_scores), len(GRU_scores), len(VAR_scores), len(target))

In [ ]:
scores = {
    'OLS': OLS_scores,
    'LSTM': LSTM_scores,
    'RETAIN': RETAIN_scores,
    'GRU': GRU_scores,
    'VAR': VAR_scores
}

final_score = pd.DataFrame(scores, index = target)
final_score['recommended model'] = final_score.idxmin(axis=1)
final_score